Design and implement a data structure for a Least Frequently Used (LFU) cache.

Implement the LFUCache class:

LFUCache(int capacity) Initializes the object with the capacity of the data structure.
int get(int key) Gets the value of the key if the key exists in the cache. Otherwise, returns -1.
void put(int key, int value) Update the value of the key if present, or inserts the key if not already present. When the cache reaches its capacity, it should invalidate and remove the least frequently used key before inserting a new item. For this problem, when there is a tie (i.e., two or more keys with the same frequency), the least recently used key would be invalidated.
To determine the least frequently used key, a use counter is maintained for each key in the cache. The key with the smallest use counter is the least frequently used key.

When a key is first inserted into the cache, its use counter is set to 1 (due to the put operation). The use counter for a key in the cache is incremented either a get or put operation is called on it.

The functions get and put must each run in O(1) average time complexity.

 

Example 1:

Input
["LFUCache", "put", "put", "get", "put", "get", "get", "put", "get", "get", "get"]
[[2], [1, 1], [2, 2], [1], [3, 3], [2], [3], [4, 4], [1], [3], [4]]
Output
[null, null, null, 1, null, -1, 3, null, -1, 3, 4]

Explanation
// cnt(x) = the use counter for key x
// cache=[] will show the last used order for tiebreakers (leftmost element is  most recent)
LFUCache lfu = new LFUCache(2);
lfu.put(1, 1);   // cache=[1,_], cnt(1)=1
lfu.put(2, 2);   // cache=[2,1], cnt(2)=1, cnt(1)=1
lfu.get(1);      // return 1
                 // cache=[1,2], cnt(2)=1, cnt(1)=2
lfu.put(3, 3);   // 2 is the LFU key because cnt(2)=1 is the smallest, invalidate 2.
                 // cache=[3,1], cnt(3)=1, cnt(1)=2
lfu.get(2);      // return -1 (not found)
lfu.get(3);      // return 3
                 // cache=[3,1], cnt(3)=2, cnt(1)=2
lfu.put(4, 4);   // Both 1 and 3 have the same cnt, but 1 is LRU, invalidate 1.
                 // cache=[4,3], cnt(4)=1, cnt(3)=2
lfu.get(1);      // return -1 (not found)
lfu.get(3);      // return 3
                 // cache=[3,4], cnt(4)=1, cnt(3)=3
lfu.get(4);      // return 4
                 // cache=[4,3], cnt(4)=2, cnt(3)=3
 

Constraints:

1 <= capacity <= 104
0 <= key <= 105
0 <= value <= 109
At most 2 * 105 calls will be made to get and put.


In [1]:
from collections import defaultdict, OrderedDict

class Node:
    def __init__(self, key, val):
        self.key = key
        self.val = val
        self.freq = 1
        self.prev = self.next = None

class DoublyLinkedList:
    def __init__(self):
        self.head = Node(None, None)  # dummy head
        self.tail = Node(None, None)  # dummy tail
        self.head.next = self.tail
        self.tail.prev = self.head

    def insert_front(self, node):
        node.next = self.head.next

        node.prev = self.head
        node.next.prev = node
        
        self.head.next = node

    def remove(self, node):
        prev_node = node.prev
        next_node = node.next
        prev_node.next = next_node
        next_node.prev = prev_node

    def pop_tail(self):
        if self.tail.prev == self.head:
            return None
        tail_node = self.tail.prev
        self.remove(tail_node)
        return tail_node

    def is_empty(self):
        return self.head.next == self.tail


In [ ]:
from collections import defaultdict

class LFUCache:

    def __init__(self, capacity: int):
        self.capacity = capacity
        self.key_to_node = {}                      # key -> node
        self.freq_to_dll = defaultdict(DoublyLinkedList)  # freq -> DoublyLinkedList
        self.min_freq = 0

    def _update_freq(self, node):  
        # get the frequency
        freq = node.freq

        # remove it from the freq map,
        self.freq_to_dll[freq].remove(node)

        # update the min frequency.
        # when the min frequncy's dll is empty, means we are done with it, so move on.
        if freq == self.min_freq and self.freq_to_dll[freq].is_empty():
            self.min_freq += 1

        # and add it in the updated (increased +1) frequency.
        node.freq += 1
        self.freq_to_dll[freq + 1].insert_front(node)

    def get(self, key: int ):
        if key not in self.key_to_node:
            return -1
        
        node = self.key_to_node[key]
        self._update_freq(node)
        return node.val
    
    def put(self, key:int, value: int) -> None:
        if self.capacity == 0:
            return
        
        # case 1: updating the key.
        if key in self.key_to_node:
            node = self.key_to_node(key)
            node.value = value
            self._update_freq(node)
            return
        
        # case 3: if capacity exceeds then remove the LRC of LFU.
        if len(self.key_to_node) >= self.capacity:
            # remove the node from the 
            dll = self.freq_to_dll[self.min_freq]
            evict_node = dll.pop_tail()
            del self.key_to_node[evict_node.key]

        # case 2: inserting the key.
        new_ndoe = Node(key, value)
        self.freq_to_dll[0].insert_front(new_ndoe)
        self.key_to_node[key] = new_ndoe
        self.min_freq = 1
